In [31]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
#from bertopic import BERTopic
import pandas as pd
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from langdetect import detect, DetectorFactory, LangDetectException


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Braider\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
rel_path = 'data/polarity_subjectivity_data_without_LemmStemm.csv'
csv_path = os.path.join(os.getcwd(), rel_path)
df_transcripts = pd.read_csv(csv_path)
df_transcripts

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\n \n [audience cheering and applauding] \n He...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0000,0.504545
1,1,https://scrapsfromtheloft.com/comedy/jeff-dunh...,"\n \n I’m funnier than he is, but they told me...",Jeff Dunham,4943,1,0.062,0.673,0.264,1.0000,0.535705
2,2,https://scrapsfromtheloft.com/comedy/taylor-to...,\n \n In her 2024 Netflix stand-up comedy spec...,Taylor Tomlinson,11197,2,0.077,0.717,0.206,1.0000,0.510567
3,3,https://scrapsfromtheloft.com/comedy/kevin-bri...,\n \n Kevin Bridges: The Overdue Catch-Up (20...,Kevin Bridges,12777,3,0.087,0.796,0.117,0.9999,0.504952
4,4,https://scrapsfromtheloft.com/comedy/jacquelin...,"\n \n In “Get on Your Knees,” Jacqueline Nova...",Jacqueline Novak,17783,4,0.077,0.733,0.190,1.0000,0.535544
...,...,...,...,...,...,...,...,...,...,...,...
468,468,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Jammin’ in New York is George Carlin’s 1...,George Carlin,8298,468,0.132,0.756,0.112,-0.9998,0.522932
469,469,https://scrapsfromtheloft.com/comedy/jim-jeffe...,\n \n Australian comedian Jim Jefferies breaks...,Jim Jefferies,2400,469,0.123,0.762,0.115,-0.9935,0.530056
470,470,https://scrapsfromtheloft.com/comedy/reggie-wa...,"\n \n Hello, I’m Thomas. I’m so glad to meet y...",Reggie Watts,4428,470,0.085,0.707,0.208,1.0000,0.562398
471,471,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Complaints and Grievances is a HBO stan...,George Carlin,8173,471,0.121,0.756,0.123,-0.9954,0.494151


## LDA

In [53]:
# Removing records that are not in english
DetectorFactory.seed = 0

def det_lang(transcript):
    try:
        return detect(transcript)
    except LangDetectException:
        return None
    
df_transcripts['language'] = df_transcripts['transcript'].apply(det_lang)

final_df = df_transcripts[df_transcripts['language'] == 'en']

final_df

,Unnamed: 0,URL,transcript,Comedian Name,word_count,Unique ID,neg_polarity,neu_polarity,pos_polarity,compound,subjectivity,language
0,0,https://scrapsfromtheloft.com/comedy/pete-holm...,\n \n [audience cheering and applauding] \n He...,Pete Holmes,10068,0,0.056,0.688,0.256,1.0000,0.504545,en
1,1,https://scrapsfromtheloft.com/comedy/jeff-dunh...,"\n \n I’m funnier than he is, but they told me...",Jeff Dunham,4943,1,0.062,0.673,0.264,1.0000,0.535705,en
2,2,https://scrapsfromtheloft.com/comedy/taylor-to...,\n \n In her 2024 Netflix stand-up comedy spec...,Taylor Tomlinson,11197,2,0.077,0.717,0.206,1.0000,0.510567,en
3,3,https://scrapsfromtheloft.com/comedy/kevin-bri...,\n \n Kevin Bridges: The Overdue Catch-Up (20...,Kevin Bridges,12777,3,0.087,0.796,0.117,0.9999,0.504952,en
4,4,https://scrapsfromtheloft.com/comedy/jacquelin...,"\n \n In “Get on Your Knees,” Jacqueline Nova...",Jacqueline Novak,17783,4,0.077,0.733,0.190,1.0000,0.535544,en
...,...,...,...,...,...,...,...,...,...,...,...,...
468,468,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Jammin’ in New York is George Carlin’s 1...,George Carlin,8298,468,0.132,0.756,0.112,-0.9998,0.522932,en
469,469,https://scrapsfromtheloft.com/comedy/jim-jeffe...,\n \n Australian comedian Jim Jefferies breaks...,Jim Jefferies,2400,469,0.123,0.762,0.115,-0.9935,0.530056,en
470,470,https://scrapsfromtheloft.com/comedy/reggie-wa...,"\n \n Hello, I’m Thomas. I’m so glad to meet y...",Reggie Watts,4428,470,0.085,0.707,0.208,1.0000,0.562398,en
471,471,https://scrapsfromtheloft.com/comedy/george-ca...,\n \n Complaints and Grievances is a HBO stan...,George Carlin,8173,471,0.121,0.756,0.123,-0.9954,0.494151,en


In [54]:
# Creating document term matrix 

cust_stop_words = ['goes', 'how', 'know', 'oh', 'got', 'said', 'like', 'just', 'right', 'we', 'got', 'going', 'go', 'get', 'one', 'yeah'
                   'gonna', 'okay']

eng_stop_words = stopwords.words('english')

total_stp_words = eng_stop_words + cust_stop_words
count_vectorizer = CountVectorizer(stop_words= total_stp_words)
count_data = count_vectorizer.fit_transform(final_df['transcript'])
df_tfd = pd.DataFrame(count_data.toarray(), columns = count_vectorizer.vocabulary_)
df_tfd.index = final_df['Comedian Name']
df_tfd


,audience,cheering,applauding,hello,applause,continue,thank,guys,much,minneapolis,...,condescended,patronized,goeth,suez,delusions,kufi,refuge,stipulations,amend,internment
Comedian Name,,,,,,,,,,,,,,,,,,,,,
Pete Holmes,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jeff Dunham,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Taylor Tomlinson,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kevin Bridges,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jacqueline Novak,0,6,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
George Carlin,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jim Jefferies,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Reggie Watts,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:

# Number of topics to extract
number_of_topics = 5

# Create and fit the LDA model
lda = LatentDirichletAllocation(n_components=number_of_topics, random_state = 55)
lda.fit(df_tfd)

# Function to display topics
def print_topics(model, vectorizer, num_top_words):
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f'Topic {topic_idx + 1}:')
        print(" ".join([words[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

# Number of words to display per topic
num_top_words = 10
print_topics(lda, count_vectorizer, num_top_words)

Topic 1:
crowd fucking dad mate wee bit guy man shite son
Topic 2:
cking lai rowan ck expletive plays americans bam drum hussein
Topic 3:
audience laughter laughing laughs people yeah think cheering time would
Topic 4:
people think yeah would gonna want time say really good
Topic 5:
shit man fuck gonna fucking people see want yeah cause


## NMF

In [47]:
# Create and fit the NMF model
number_of_topics = 5

nmf = NMF(n_components=number_of_topics, random_state=0)
nmf.fit(df_tfd)

# Function to display topics
def print_topics(model, vectorizer, num_top_words):
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f'Topic {topic_idx + 1}:')
        print(" ".join([words[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

# Number of words to display per topic
num_top_words = 10
print_topics(nmf, count_vectorizer, num_top_words)

Topic 1:
people think yeah would want gonna time really good say
Topic 2:
laughing audience applauding cheering crowd think little yeah time chuckles
Topic 3:
laughter applause yeah cheering people cheers fucking guy voice hey
Topic 4:
laughs audience crowd yeah chuckles people cheers cause want think
Topic 5:
shit fuck man fucking gonna ass white come see cause


## LSA

In [51]:
number_of_topics = 5
num_top_words = 10
# Create and fit the LSA model (also known as SVD in this context)
lsa = TruncatedSVD(n_components=number_of_topics, random_state=0)
lsa.fit(df_tfd)

def print_topics(model, vectorizer, num_top_words):
    words = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        print(f'Topic {topic_idx + 1}:')
        print(" ".join([words[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

print_topics(lsa, count_vectorizer, num_top_words)

Topic 1:
audience people gonna yeah laughing shit think man fucking laughter
Topic 2:
audience laughing laughs applauding cheering crowd chuckles continues clapping chuckling
Topic 3:
laughter applause audience laughs cheering cheers yeah voice chuckles mom
Topic 4:
laughs audience cheers want people applauds uh guys guy would
Topic 5:
shit fuck man fucking laughs motherfucker ass white black bitch
